In [85]:
#!pip install selenium
#!pip install bs4

  Could not find a version that satisfies the requirement urlparse (from versions: )
No matching distribution found for urlparse


In [179]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import InvalidElementStateException
import time
import pprint
import re
import csv

In [167]:
BASE_URL = 'https://scholar.google.com/citations'

In [216]:
def get_users(): 
    
    #links from spreadsheet, next time we can grab 
    sla_authors = ['https://scholar.google.com/citations?user=v9rIqPQAAAAJ&hl=en',
    'https://scholar.google.com/citations?hl=en&user=igIKOZcAAAAJ&view_op=list_works&sortby=pubdate',
    'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=timothy+malloy+ucla&btnG=',
    'https://scholar.google.com/citations?user=gkygJ18AAAAJ&hl=en&oi=ao',
    'https://scholar.google.com/citations?user=1AG9uzsAAAAJ&hl=en',
    'https://scholar.google.com/citations?user=Rl5xhgwAAAAJ&hl=en&oi=ao',
    'https://scholar.google.com/citations?user=AfHMFccAAAAJ&hl=en&oi=ao',
    'https://scholar.google.com/citations?user=VXvek_AAAAAJ&hl=en&oi=sra',
    'https://scholar.google.com/citations?user=9CC9gHoAAAAJ&hl=en&oi=ao',
    'https://scholar.google.com/citations?user=rOvEiAYAAAAJ&hl=en',
    'https://scholar.google.com/citations?user=GVYNtNsAAAAJ&hl=en',
    'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=CW+Wong+ucla&btnG=',
    'https://scholar.google.com/citations?user=ceCfTvcAAAAJ&hl=en&oi=ao',
    'https://scholar.google.com/citations?user=kvSIKM8AAAAJ&hl=en&oi=ao']

    user_id_regex = re.compile("user=(.{12})&")

    user_id = [m.group(1) for a in sla_authors for m in [user_id_regex.search(a)] if m]
    #[au in au for sla_authors]

    #print(user_id)
    return user_id

In [218]:
def get_full_user_page(user):
    """
    Will click to show all of the articles.
    
    :param user: an ID for a google scholar user
    :returns: returns a list of articles
    
    """
    
    url = BASE_URL + "?user=" + user
    
    driver.get(url)

    pages_remaining = True

    while pages_remaining:

        try:
            #Checks if there are more pages with links
            driver.find_element_by_id('gsc_bpf_more').click()         
            time.sleep(1)
            #print('pages_remaining ', pages_remaining)
        except InvalidElementStateException:
            gscholar_citations = driver.page_source
            pages_remaining = False
            #print('pages_remaining ', pages_remaining)
    
    #driver.quit()
    
    return gscholar_citations


In [208]:
def get_author_citations_links(page):
    """
    Will take a google sholar profile page and convert to BeautifulSoup object, 
    then extract the author & citation links. Then returns as a list. 
    
    :param page: a google scholar author page
    :returns: returns a list that contains the name of the author and the citation links
    
    """
    #make a soup object
    gscholar_soup = BeautifulSoup(page, 'lxml')
    
   
    author = gscholar_soup.find(id="gsc_prf_in").get_text()
    #author_cite_links.append(author)
    citation_links = [link['data-href'] for link in gscholar_soup.select("a.gsc_a_at")]
    
    #get the IDs from the author_cite_lists 
    citation_id_regex = re.compile("citation_for_view=(.{25})")
    citation_ids = [m.group(1) for c in citation_links for m in [citation_id_regex.search(c)] if m]
    
    return author, citation_ids

In [209]:
author, citation_list = get_author_citations_links(page)

In [303]:
get_citations(author, citation_list[:3])

[{'Authors': 'N Shyamsundar, Rajit Gadh',
  'Issue': '9',
  'Journal': 'Computer-aided design',
  'Pages': '637-651',
  'Publication date': '2001/8/1',
  'Volume': '33',
  'sla_author': 'Rajit Gadh',
  'title': 'Internet-based collaborative product design with assembly features and virtual design spaces',
  'url': 'https://www.sciencedirect.com/science/article/pii/S0010448501000690'},
 {'Authors': 'N Shyamsundar, Rajit Gadh',
  'Issue': '10',
  'Journal': 'Computer-Aided Design',
  'Pages': '755-768',
  'Publication date': '2002/9/1',
  'Volume': '34',
  'sla_author': 'Rajit Gadh',
  'title': 'Collaborative virtual prototyping of product assemblies over the Internet',
  'url': 'https://www.sciencedirect.com/science/article/pii/S0010448501002044'},
 {'Application number': '09409569',
  'Inventors': 'Rajit Gadh, Tushar H Dani',
  'Patent number': '6629065',
  'Patent office': 'US',
  'Publication date': '2003/9/30',
  'sla_author': 'Rajit Gadh',
  'title': 'Methods and apparata for rapid

In [300]:
citation_list

['v9rIqPQAAAAJ:u5HHmVD_uO8C',
 'v9rIqPQAAAAJ:u-x6o8ySG0sC',
 'v9rIqPQAAAAJ:d1gkVwhDpl0C',
 'v9rIqPQAAAAJ:9yKSN-GCB0IC',
 'v9rIqPQAAAAJ:2osOgNQ5qMEC',
 'v9rIqPQAAAAJ:UeHWp8X0CEIC',
 'v9rIqPQAAAAJ:qjMakFHDy7sC',
 'v9rIqPQAAAAJ:LkGwnXOMwfcC',
 'v9rIqPQAAAAJ:IjCSPb-OGe4C',
 'v9rIqPQAAAAJ:zYLM7Y9cAGgC',
 'v9rIqPQAAAAJ:Y0pCki6q_DkC',
 'v9rIqPQAAAAJ:Tyk-4Ss8FVUC',
 'v9rIqPQAAAAJ:W7OEmFMy1HYC',
 'v9rIqPQAAAAJ:YsMSGLbcyi4C',
 'v9rIqPQAAAAJ:eQOLeE2rZwMC',
 'v9rIqPQAAAAJ:lmc2jWPfTJgC',
 'v9rIqPQAAAAJ:WF5omc3nYNoC',
 'v9rIqPQAAAAJ:3fE2CSJIrl8C',
 'v9rIqPQAAAAJ:0EnyYjriUFMC',
 'v9rIqPQAAAAJ:ufrVoPGSRksC',
 'v9rIqPQAAAAJ:_FxGoFyzp5QC',
 'v9rIqPQAAAAJ:Se3iqnhoufwC',
 'v9rIqPQAAAAJ:UebtZRa9Y70C',
 'v9rIqPQAAAAJ:roLk4NBRz8UC',
 'v9rIqPQAAAAJ:5nxA0vEk-isC',
 'v9rIqPQAAAAJ:hqOjcs7Dif8C',
 'v9rIqPQAAAAJ:pqnbT2bcN3wC',
 'v9rIqPQAAAAJ:MXK_kJrjxJIC',
 'v9rIqPQAAAAJ:8k81kl-MbHgC',
 'v9rIqPQAAAAJ:kNdYIx-mwKoC',
 'v9rIqPQAAAAJ:Zph67rFs4hoC',
 'v9rIqPQAAAAJ:KlAtU1dfN6UC',
 'v9rIqPQAAAAJ:YOwf2qJgpHMC',
 'v9rIqPQA

In [304]:
def get_citations(author, cids):
    """
    Takes a list that contains the author and her citation page links. Opens and scrapes
    citation pages and builds a dictionary of the author and citations. Returns the dict.
    
    :param links: a list containing author and another list of citation pages.
    :returns: returns a dictionary with the author and their citations.
    """
        
    author_bib = []
    
    for l in cids: 
        #citation_for_view=igIKOZcAAAAJ:tntj4plCNvAC
        search_params = {'view_op': 'view_citation',
                         'h1': 'en',
                         'citation_for_view':l}
        cite_page = requests.get(BASE_URL, params=search_params)
        #print(cite_page.url)
        cpage_soup = BeautifulSoup(cite_page.text, 'lxml')
        #link & title = a gsc_vcd_title_link
        #authors = div gsc_vcd_value
        #date = div gsc_vcd_value
        citation = {}
        citation['title'] = cpage_soup.select_one("a.gsc_vcd_title_link").text
        citation['url'] = cpage_soup.select_one("a.gsc_vcd_title_link")['href']
        citation['sla_author'] = author
        for ele in cpage_soup.select('div.gsc_vcd_field'):
            if ele.text in ["Description", "Total citations", "Scholar articles", "Publisher", "Application number"]:
                continue
            else: 
                citation[ele.text] = ele.next_sibling.text
        author_bib.append(citation)
    return author_bib

In [305]:
def write_out_csv(author, citations):
    pass
    

### run the code below
1. start webdriver, it will stay active till you driver.quit() 
  * TODO: run headless 
2. run loop that will grab uids from list

In [230]:
driver = webdriver.Firefox()

In [231]:
for uid in get_users()[:1]:
    print(uid)
    page = get_full_user_page(uid)
    #show_all_articles on profile page
    #full_page = show_all_articles(uid)
    #sla_author_name, citations = get_citations(get_author_citations_links(full_page))
    #TODO
    #save out in csv

v9rIqPQAAAAJ


str

In [220]:
driver.quit()

SessionNotCreatedException: Message: Tried to run command without establishing a connection


In [213]:
import re
regex = re.compile('th.s')
l = ['this', 'is', 'just', 'a', 'test']

In [214]:
matches = [string for string in l if re.match(regex, string)]

In [215]:
matches

['this']

In [291]:
search_params = {'view_op': 'view_citation',
                         'h1': 'en',
                         'citation_for_view':'igIKOZcAAAAJ:tntj4plCNvAC'}
cpage = requests.get(BASE_URL, params=search_params)

In [286]:
cpage = requests.get('https://scholar.google.com/citations?view_op=view_citation&hl=en&citation_for_view=igIKOZcAAAAJ:tntj4plCNvAC')

In [292]:
cpage.url

'https://scholar.google.com/citations?citation_for_view=igIKOZcAAAAJ%3Atntj4plCNvAC&view_op=view_citation&h1=en'

In [293]:
c_page_soup = BeautifulSoup(cpage.text, 'lxml')

In [294]:
c_page_soup.select('a.gsc_vcd_title_link')[0].text

'Ge/Si nanowire heterostructures as high-performance field-effect transistors'

In [295]:
c_page_soup.select_one('a.gsc_vcd_title_link')['href']

'https://www.nature.com/articles/nature04796'

In [ ]:
https://scholar.google.com/citations?view_op=view_citation&hl=en&user=igIKOZcAAAAJ&citation_for_view=igIKOZcAAAAJ:tntj4plCNvAC